In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
count=CountVectorizer()

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
data = pd.read_csv("/gdrive/My Drive/Esha/Project/Suicide_Detection.csv")
data


In [ ]:
data["class"]

In [ ]:
type(data["Unnamed: 0"][3])

In [ ]:
text = data['text']
label = data['class']

In [ ]:
text[0]

In [ ]:
classes_list = ["non-suicide", "suicide"]
label_index = data['class'].apply(classes_list.index)
label = np.asarray(label_index)

In [ ]:
data['label'] = label

In [ ]:
data

In [ ]:
data.drop("class", axis = 1, inplace  =True)
data

In [ ]:
data["label"].value_counts()

In [ ]:
fig=plt.figure(figsize=(5,5))
colors=["skyblue",'pink']
pos=data[data['label']==1]
neg=data[data['label']==0]
ck=[pos['label'].count(),neg['label'].count()]
legpie=plt.pie(ck,labels=["suicide","non-suicide"],
                 autopct ='%1.1f%%', 
                 shadow = True,
                 colors = colors,
                 startangle = 45,
                 explode=(0, 0.1))

In [ ]:
import re
def preprocessor(text):
             text=re.sub('<[^>]*>','',text)
             emojis=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
             text=re.sub('[\W]+',' ',text.lower()) +\
                ' '.join(emojis).replace('-','')
             return text   
data['text']=data['text'].apply(preprocessor)

In [ ]:
data

In [ ]:
data = data[:10000]

In [ ]:
data["label"].value_counts()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# for i in range(len(data['text'])):
#     data['text'][i]  =data['text'][i].lower()

In [ ]:
data

In [ ]:
from nltk.stem.porter import PorterStemmer
porter=PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
for i in range(len(data['text'])):
    data['text'][i]  =  np.array(tokenizer_porter(data['text'][i]))

In [ ]:
data

In [ ]:
data["text"][0]

In [ ]:
print(data["text"][0])

In [ ]:
#data.to_csv("filtered_data.csv")

In [ ]:
#data = pd.read_csv("/gdrive/filtered_data.csv")

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stops=stopwords.words('english')
nltk.download('stopwords')

In [ ]:
def remove_stopwords(lower_tokens):
  filtered_words=[]
  for s in lower_tokens:
    temp=[]
    for token in s:
      if token not in stops:
        temp.append(token)
    filtered_words.append(temp)
  return filtered_words


In [ ]:
t_list = remove_stopwords(data['text'])
data['text'] = t_list

In [ ]:
for i in range(len(data['text'])):
    data['text'][i]  =  " ".join(data['text'][i])

In [ ]:
data

In [ ]:
data["text"][0]

In [ ]:
#data.to_csv("filtered1_data.csv")
#data = pd.read_csv("filtered1_data.csv")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
y =data['label'].values
tfidf = vectorizer.fit(data['text'])
x = vectorizer.transform(data["text"])

In [ ]:
tfidf.vocabulary_

In [ ]:
import pickle

pickle.dump(tfidf, open("tfidf.pickle", "wb"))
pickle.dump(x, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,random_state=1,test_size=0.3,shuffle=False)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500)
classifier = clf.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
cm=metrics.confusion_matrix(y_test,y_pred)
print(cm)

In [ ]:
import pickle
pickle.dump(classifier, open("lr_model.sav", 'wb'))
 
# some time later...
 
# load the model from disk
import pickle
loaded_model = pickle.load(open("lr_model.sav", 'rb'))
result = loaded_model.score(X_test, y_test)

In [ ]:
result

# Deep learning model

In [ ]:
x.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model = Sequential()
#model.add(LSTM(128,  input_shape=(20307, 20307),return_sequences=True))
model.add(Dense(3, activation='relu', input_shape=(20307,)))
#model.add(Dropout(.4))
#model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
model.fit(X_train.toarray(), y_train, validation_split =.2, epochs=30, verbose=1)

In [ ]:
y_pred = model.predict(X_test.toarray())

In [ ]:
y_pred

In [ ]:
y_pred = (y_pred>.5)*1

In [ ]:
cm=metrics.confusion_matrix(y_test,y_pred)
print(cm)

In [ ]:
(cm[1, 1] + cm[0, 0])/sum(sum(cm))

In [ ]:
# 

# deployement part

In [ ]:
text = input("Enter a text to check if it is suicidal: ")

from nltk.stem.porter import PorterStemmer
porter=PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

text = np.array(tokenizer_porter(text))

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stops=stopwords.words('english')
nltk.download('stopwords')


def remove_stopwords(lower_tokens):
  filtered_words=[]
  for s in lower_tokens:
    temp=[]
    for token in s:
      if token not in stops:
        temp.append(token)
    filtered_words.append(temp)
  return filtered_words


f_text = np.array(remove_stopwords([text])[0])




f_text = " ".join(f_text)


import pickle

tfidf_vectorizer = pickle.load(open("tfidf.pickle", "rb"))
ss2 = tfidf_vectorizer.transform([f_text])

# load the model from disk
import pickle
loaded_model = pickle.load(open("lr_model.sav", 'rb'))
result = loaded_model.predict(ss2)

In [ ]:
result[0]

In [ ]:
tfidf

In [ ]:
corpus = np.array([])

In [ ]:
X_train.shape

In [ ]:
# create the model
model = Sequential()
#model.add(Embedding(top_words, 32, input_length=max_words))
#model.add(Conv1D(32, 3, padding='same', activation='relu'))
#model.add(MaxPooling1D())
model.add(LSTM(64))
#model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()